<a href="https://colab.research.google.com/github/ampham03/grad-assessment/blob/main/grad_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from transformers import MarianMTModel, MarianTokenizer
from datasets import load_dataset

In [3]:
model_name = "Helsinki-NLP/opus-mt-en-de"  # English to German
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)

# load WMT dataset for de-en
de_en_dev = load_dataset("wmt19", "de-en", split='validation')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

train-00014-of-00016.parquet:   0%|          | 0.00/305M [00:00<?, ?B/s]

train-00015-of-00016.parquet:   0%|          | 0.00/231M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/495k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/34782245 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2998 [00:00<?, ? examples/s]

In [4]:
def tokenize_sentence(sentence):
    return tokenizer(sentence, return_tensors="pt", padding=True).input_ids

In [39]:
class Hypothesis:
    def __init__(self, score=0, is_open=True, seq=None):
      self.score = score
      self.is_open = is_open
      self.seq = seq


In [6]:
def init_grid(max_len, num_constraints):
  grid = []
  for i in range(max_len):
    row = []
    for j in range(num_constraints + 1):
        row.append([])
    grid.append(row)
  return grid

In [43]:
# generate new open hypotheses
def generate(model, hyp, input_ids):

    # forward pass
    with torch.no_grad():
        encoder_outputs = model.get_encoder()(input_ids=input_ids)

    # initialize decoder input
    if hyp.seq is None:
        decoder_input_ids = torch.tensor([[model.config.decoder_start_token_id]])
    else:
        decoder_input_ids = hyp.seq

    with torch.no_grad():
      outputs = model(
          input_ids=None,
          encoder_outputs=encoder_outputs,
          decoder_input_ids=decoder_input_ids
      )
      logits = outputs.logits

    # get the logits for the last predicted token
    next_token_logits = logits[:, -1, :] # [batch_size, sequence_length, vocab_size]
    next_token_probs = torch.softmax(next_token_logits, dim=-1).squeeze(0)

    # get the token with the highest probability
    next_token_id = torch.argmax(next_token_probs).item()
    next_token_prob = next_token_probs[next_token_id].item()

    new_seq = torch.cat([decoder_input_ids, torch.tensor([[next_token_id]])], dim=-1)

    is_open = next_token_id != model.config.eos_token_id # check for EOS token

    new_hyp = Hypothesis(hyp.score + next_token_prob, is_open, new_seq)

    return new_hyp


In [ ]:
# start new constrained hypotheses
def start():
  pass

In [ ]:
# continue unfinished hypotheses
def cont():
  pass

In [50]:
input_ids = tokenize_sentence(de_en_dev[0]['translation']['en'])
print(generate(model, generate(model, Hypothesis(), input_ids), input_ids).seq)

tensor([[58100,  3814,   268]])


In [51]:
tokenizer.decode(torch.tensor([[58100,  3814,   268]]).flatten(), skip_special_tokens=True)

'München 18'

In [30]:
def constrained_beam_search(model, input_ids, constraints, max_len, num_constraints, beam_size):

  """
  model: model
  input: example tokenized sentence
  constraints: constraints
  max_len: max_len of translation
  num_constraints: # of constraints
  beam_size: beam_size
  """
  start_hyp = Hypothesis() # initial hypothesis
  grid = init_grid(max_len, num_constraints) # initialize beams in grid
  grid[0][0] = [start_hyp]

  for t in range(1, max_len):
    for c in range(max(0, (num_constraints + t) - max_len), min(t, num_constraints) + 1):
      n, s, g = [], [], []

      for hyp in grid[t - 1][c]:
        if hyp.is_open:
          g.append(generate(model, hyp, input_ids))

      if c > 0:
        for hyp in grid[t - 1][c - 1]:
          if hyp.is_open:
            # n.append(start())
            pass
          else:
            # s.append(cont())
            pass

      # grid[t][c] =  # k-best scoring hypotheses stay on the beam

  # top_level_hyps = grid[][num_constraints]
  # finished_hyps =
  # best_hyp =
  print(grid)
  #return best_hyp


In [31]:
input_ids = tokenize_sentence(de_en_dev[0]['translation']['en'])
constrained_beam_search(model, input_ids, ["München"], 30, 1, 2)

München
[[[<__main__.Hypothesis object at 0x7dca522a9f60>], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []], [[], []]]


In [ ]:
# compute score using BLEU
def compute_score(prediction, actual):
  pass

In [9]:
de_en_dev[0]

{'translation': {'de': 'München 1856: Vier Karten, die Ihren Blick auf die Stadt verändern',
  'en': 'Munich 1856: Four maps that will change your view of the city'}}

In [10]:
tokenize_sentence(de_en_dev[0]['translation']['en'])

tensor([[ 4260,   268,  3441,    16,  9701, 12126,    35,    73,   866,    83,
           704,     7,     4,   642,     0]])